<a href="https://colab.research.google.com/github/jeff-ai-ml/genai-langgraph/blob/main/Agent_2_With_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain langgraph openai langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 7.6 MB/s eta 0:00:00


In [5]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from google.colab import userdata

In [6]:
class AgentState(TypedDict):
  message : List[Union[HumanMessage, AIMessage]]

In [7]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=userdata.get("OPENAI_API_KEY"))

In [8]:
def process(state: AgentState) -> AgentState:
  response = llm.invoke(state["message"])
  state["message"].append(AIMessage(content=response.content))
  print(f"\nAI: {response.content}")
  return state


In [9]:
graph = StateGraph(AgentState)

graph.add_node("process node", process)
graph.add_edge(START, "process node")
graph.add_edge("process node", END)
graph.set_entry_point("process node")

agent = graph.compile()

In [10]:
conversation_history = []

In [11]:
user_input = input("Enter: ")

while user_input != "exit":
  conversation_history.append(HumanMessage(content=user_input))
  result = agent.invoke({"message": conversation_history})
  conversation_history = result["message"]
  user_input = input("Enter: ")



Enter: hello

AI: Hello! How can I assist you today?
Enter: I am Jeff, your name please

AI: Nice to meet you, Jeff! I’m an AI language model and don’t have a personal name, but you can call me Assistant. How can I help you today?
Enter: what is color of sky?

AI: The color of the sky appears blue during the day due to a phenomenon called Rayleigh scattering. This occurs because the shorter blue wavelengths of sunlight are scattered in all directions by the gases and particles in the Earth's atmosphere. As the sun sets or rises, the sky can take on shades of orange, pink, and red due to the angle of the sunlight traveling through more of the atmosphere, scattering the shorter wavelengths and allowing longer wavelengths to predominate. At night, the sky can appear black or dark blue, showcasing stars and celestial bodies.
Enter: do you know me?

AI: I don’t have personal knowledge or memories, so I don’t know you personally. I only know what you share with me in our conversation. How ca

In [12]:
with open("logging.txt", "w") as file:

  file.write("Your conversation log: \n")

  for message in conversation_history:
    if isinstance(message, HumanMessage):
      file.write(f"You: {message.content}\n")
    elif isinstance(message, AIMessage):
      file.write(f"AI: {message.content}\n\n")

  file.write("End of coversation \n")

print("Conversation is saved to the file logging.txt")



Conversation is saved to the file logging.txt
